In [1]:
import os
import pandas as pd

In [2]:
train_path = "../data_ready/zero_shot.csv"
print(f"zero_shot data exist? : {os.path.exists(train_path)}")

data=pd.read_csv(train_path).sample(frac=1)#.reset_index() #shuffle dataframe

data_text, data_lables = data['text'], data['class']
possible_data_labels = ['exclusive','inclusive']

print(data_text.head())

zero_shot data exist? : True
42                       Italians are loud and romantic.
278    Les cultures autochtones ont beaucoup d'histoi...
117           Les Asiatiques sont timides et silencieux.
324    La culture singapourienne est un creuset de di...
331    Le désert du Sahara n'est pas seulement un dés...
Name: text, dtype: object


In [3]:
import transformers
# import tensorrt
classifier = transformers.pipeline("zero-shot-classification", model="morit/french_xlm_xnli")
zero_shot_results = classifier(list(data_text), possible_data_labels)

/home/blake/venvs/fyp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-26 15:59:17.484922: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 15:59:17.920947: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 15:59:17.920973: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on you

In [5]:
zero_shot_results

text = list()
inclusive_score = list()
exclusive_score = list()

for result in zero_shot_results:
  text.append(result.get("sequence"))
  inclusive_score.append(result.get('scores')[0])
  exclusive_score.append(result.get('scores')[1])

results_formatted = pd.DataFrame()
results_formatted['text'] = text
results_formatted['inclusive'] = inclusive_score
results_formatted['exclusive'] = exclusive_score

results_formatted.to_csv("../results/french_xlm_xnli.csv", index=False)